<a href="https://colab.research.google.com/github/wooridle/mlp_lecture/blob/main/custom_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms
import torch.nn.functional as F



class CustomImageDataset(Dataset):
    def read_data_set(self):

        all_img_files = []
        all_labels = []

        class_names = os.walk(self.data_set_path).__next__()[1][1:]

        for index, class_name in enumerate(class_names):
            label = index
            img_dir = os.path.join(self.data_set_path, class_name)
            img_files = os.walk(img_dir).__next__()[2]

            for img_file in img_files:
                img_file = os.path.join(img_dir, img_file)
                img = Image.open(img_file)
                if img is not None:
                    all_img_files.append(img_file)
                    all_labels.append(label)

        return all_img_files, all_labels, len(all_img_files), len(class_names)

    def __init__(self, data_set_path, transforms=None):
        self.data_set_path = data_set_path
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()
        self.transforms = transforms

    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("RGB")

        if self.transforms is not None:
            image = self.transforms(image)

        return {'image': image, 'label': self.labels[index]}

    def __len__(self):
        return self.length

hyper_param_epoch = 20
hyper_param_batch = 8
hyper_param_learning_rate = 0.001

transforms_train = transforms.Compose([transforms.Resize((32, 32)),
                                       transforms.RandomRotation(10.),
                                       transforms.ToTensor()])

transforms_test = transforms.Compose([transforms.Resize((32, 32)),
                                      transforms.ToTensor()])


train_data_set = CustomImageDataset(data_set_path="./data/train", transforms=transforms_train)
train_loader = DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True)

test_data_set = CustomImageDataset(data_set_path="./data/test", transforms=transforms_test)
test_loader = DataLoader(test_data_set, batch_size=hyper_param_batch, shuffle=True)

if not (train_data_set.num_classes == test_data_set.num_classes):
    print("error: Numbers of class in training set and test set are not equal")
    exit()

IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
COLOR_CHANNELS = 3

N_CLASSES = train_data_set.num_classes


class Net(torch.nn.Module):

    def __init__(self, n_hidden_nodes, n_hidden_layers):

        super(Net, self).__init__()

        self.n_hidden_nodes = n_hidden_nodes

        self.n_hidden_layers = n_hidden_layers

        self.fc1 = nn.Linear(IMAGE_WIDTH * IMAGE_WIDTH * COLOR_CHANNELS,
                                   n_hidden_nodes)

        self.dropout = nn.Dropout(0.5)

        self.out = nn.Linear(n_hidden_nodes, N_CLASSES)

    def forward(self, x):

        x = x.view(-1, IMAGE_WIDTH * IMAGE_WIDTH * COLOR_CHANNELS)

        x = F.relu(self.fc1(x))

        x = self.dropout(x)

        return F.log_softmax(self.out(x)) 


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_classes = train_data_set.num_classes


hidden_nodes = 100
layers = 1
#### custom_model = CustomConvNet(num_classes=num_classes).to(device)
#### H is hidden dimension; D_out is output dimension.

batch_size = 64
test_batch_size = 64
input_size = 768

N, D_in, H, D_out = batch_size, input_size, 32, 2

custom_model = Net(hidden_nodes, layers)

#### Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)



for e in range(hyper_param_epoch):

    for i_batch, item in enumerate(train_loader):

        images = images.reshape(-1, 32*32*3)

        images = item['image'].to(device)

        labels = item['label'].to(device)

        #### Forward pass

        outputs = custom_model(images)

        loss = criterion(outputs, labels)

        #### Backward and optimize

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        if (i_batch + 1) % hyper_param_batch == 0:

            print('Epoch [{}/{}], Loss: {:.4f}'
            
                  .format(e + 1, hyper_param_epoch, loss.item()))

#### Test the model

custom_model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)

with torch.no_grad():

    correct = 0

    total = 0

    for item in test_loader:

        images = item['image'].to(device)

        labels = item['label'].to(device)

        print("labels: ", labels)

        outputs = custom_model(images)
       
        _, predicted = torch.max(outputs.data, 1)

        print("predicted: ", predicted)

        total += len(labels)

        correct += (predicted == labels).sum().item()
        
        print(correct)

    print('Test Accuracy of the model on the {} test images: {} %'.format(total, 100 * correct / total))

labels:  tensor([1, 1, 0, 1, 1, 1, 0, 1])
predicted:  tensor([1, 1, 0, 1, 1, 1, 0, 1])
8
labels:  tensor([1, 0, 1, 1, 1, 0, 0, 0])
predicted:  tensor([1, 0, 1, 1, 1, 0, 0, 0])
16
labels:  tensor([0, 0, 0, 0])
predicted:  tensor([0, 0, 0, 0])
20
Test Accuracy of the model on the 20 test images: 100.0 %


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:100: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [68]:
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
COLOR_CHANNELS = 3

N_CLASSES = train_data_set.num_classes


class Net(torch.nn.Module):
    def __init__(self, n_hidden_nodes, n_hidden_layers):
        super(Net, self).__init__()
        self.n_hidden_nodes = n_hidden_nodes
        self.n_hidden_layers = n_hidden_layers

        self.fc1 = nn.Linear(IMAGE_WIDTH * IMAGE_WIDTH * COLOR_CHANNELS,
                                   n_hidden_nodes)
        self.dropout = nn.Dropout(0.5)
        self.out = nn.Linear(n_hidden_nodes, N_CLASSES)

    def forward(self, x):
        x = x.view(-1, IMAGE_WIDTH * IMAGE_WIDTH * COLOR_CHANNELS)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return F.log_softmax(self.out(x)) 


In [13]:
train_data_set[0]['image'].size()

torch.Size([3, 128, 128])

In [42]:
os.walk('./data/train').__next__()[1][1:]

['kagami', 'kuroko']